# **Introduction**

## **Chi-test (boroughs + svi)**
## **Bar-chart with svi as regression/scatterplot (boroughs first)**

source: https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documentation.pdf

source: https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

Chi-test: Using the average_year_eviction_nta_count for Chi-square test for NTA, we get results as following:

1) SVI: P-value is 0.0095 and chi-square is 6.73. We can reject the null hypothesis. The Chi-square suggests a strong-to-moderate deviation from the distribution if the two groups/categories are independent. The odds are 3.3345, meaning if you live in a nta with higher than average svi, you are 3.3345 times more likely to be evicted than living in a nta with less than average svi.

2) Black + Hispanic Population percentage: P-value is 0.00032 and chi-squiare is 53.9875. We can refute the null hypothesis. We can say we are confident that high "black+hispanic percetage" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high BH pct and high eviction rates in the full dataset. The odds are 12.5125, meaning if you live in a neighborhood with b+h population above the average pct of nyc, you are 12.5125 times more likely to get evicted than someone living in a neighborhood with below average percentage of black + hispanic people.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns

from scipy.stats import chi2_contingency
import statsmodels.api as sm

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy esda splot

In [4]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Step 1 Get the Eviction data**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# data source:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times.csv'
file_path2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid.csv'

In [7]:
evictions_pre_post_raw = pd.read_csv(file_path1)
evictions_covid_raw = pd.read_csv(file_path2)
evictions_covid_raw.shape, evictions_pre_post_raw.shape

((5386, 93), (66397, 94))

In [8]:
evictions_pre_post = evictions_pre_post_raw.copy()
evictions_covid = evictions_covid_raw.copy()

In [9]:
evictions_pre_post.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count,average_year_eviction_nta_count
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.6721,-73.8911,5.0000,37.0000,1152.0000,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8000,1930.0000,C0,3.0000,3.0000,356 MILLER LLC,2700.0000,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0000,0.9788,0.9140,0.9808,0.9812,0.9839,33.9000,11.1000,19.1000,6.0000,13.8000,22.5000,13.8000,5.3000,57.8000,9.1000,44.7000,55.9000,32.8000,1.5000,0.0000,0.0000,2.9000,1.6000,94.7000,5.3000,False,Q3,medium-high,0.0000,0.0000,1.0000,0.0000,1.0000,2.0000,0.0000,0.0000,1.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,0.0000,3.0000,5.0000,19.0000,0.2667,0.0027
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.6359,-74.0119,7.0000,38.0000,118.0000,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6000,1920.0000,B2,2.0000,2.0000,"A.R.M. PARKING, LLC",1204.0000,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0000,0.9885,0.7635,0.9594,0.9179,0.9662,37.5000,7.5000,37.9000,11.6000,13.1000,25.4000,8.4000,40.2000,61.7000,23.7000,43.6000,1.7000,40.9000,40.7000,0.4000,0.0000,1.2000,0.2000,85.0000,15.0000,False,Q3,medium-high,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,4.0000,0.3000,0.0004


In [10]:
evictions_covid.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count,average_year_eviction_nta_count
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.8654,-73.9013,7.0000,14.0000,265.0000,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),0.2000,2004.0000,C0,3.0000,3.0000,MONJU SARKER,3420.0000,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0000,0.9954,0.9407,0.9870,0.9470,0.9874,39.5000,11.6000,28.3000,9.2000,11.2000,26.4000,12.2000,26.9000,71.8000,19.2000,56.7000,15.6000,78.0000,2.3000,0.0000,0.0000,0.5000,0.5000,96.9000,3.1000,False,Q3,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,2.0000,0.0000,0.0000,0.0000,3.0000,1.0000,14.0000,0.0667,0.0001
1,0050153/20_106030,4031560133,0050153/20,106030,98-05 67TH AVENUE,12F,2022-04-14,QUEENS,11375,Not an Ejectment,Possession,40.7242,-73.8556,6.0000,29.0000,71306.0000,4074666,Forest Hills,2022,2022-04,POINT (-73.855552 40.724241),0.2000,1960.0000,D3,13.0000,181.0000,MARSEILLES LEASING LIMITED PARTNERSHIP,177710.0000,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternativ...","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,11375,75212.0000,0.4759,0.5698,0.8789,0.8057,0.7322,12.0000,4.8000,6.1000,3.7000,20.4000,18.0000,10.5000,7.9000,41.9000,5.8000,25.4000,2.7000,16.4000,28.5000,0.1000,0.0000,4.6000,0.7000,53.0000,47.0000,False,Q1 (Low),0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,0.0000,62.0000,0.0000,34.0000,0.0000,0.0000,4.0000,1.0000,0.0000,0.0000,2.0000,5.0000,112.0000,0.0011,0.0000


In [11]:
evictions_pre_post.columns, \
evictions_covid.columns, \
evictions_pre_post.shape, \
evictions_covid.shape

(Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
        'latitude', 'longitude', 'community_board', 'council_district',
        'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
        'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
        'unitsres', 'ownername', 'bldgarea', 'building_type',
        'building_category', 'is_condo', 'floor_category', 'rent_era',
        'architectural_style', 'economic_period', 'residential_units_category',
        'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
        'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
        'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
        'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
        'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp

In [12]:
link = '/content/drive/My Drive/X999/svi_cleaned.csv'

In [13]:
svi_df = pd.read_csv(link)
svi_df.head(2)

,fips,location,area_sqmi,e_totpop,m_totpop,e_hu,m_hu,e_hh,m_hh,e_pov150,m_pov150,e_unemp,m_unemp,e_hburd,m_hburd,e_nohsdp,m_nohsdp,e_uninsur,m_uninsur,e_age65,m_age65,e_age17,m_age17,e_disabl,m_disabl,e_sngpnt,m_sngpnt,e_limeng,m_limeng,e_minrty,m_minrty,e_munit,m_munit,e_mobile,m_mobile,e_crowd,m_crowd,e_noveh,m_noveh,e_groupq,m_groupq,ep_pov150,mp_pov150,ep_unemp,mp_unemp,ep_hburd,mp_hburd,ep_nohsdp,mp_nohsdp,ep_uninsur,mp_uninsur,ep_age65,mp_age65,ep_age17,mp_age17,ep_disabl,mp_disabl,ep_sngpnt,mp_sngpnt,ep_limeng,mp_limeng,ep_minrty,mp_minrty,ep_munit,mp_munit,ep_mobile,mp_mobile,ep_crowd,mp_crowd,ep_noveh,mp_noveh,ep_groupq,mp_groupq,epl_pov150,epl_unemp,epl_hburd,epl_nohsdp,epl_uninsur,spl_theme1,rpl_theme1,epl_age65,epl_age17,epl_disabl,epl_sngpnt,epl_limeng,spl_theme2,rpl_theme2,epl_minrty,spl_theme3,rpl_theme3,epl_munit,epl_mobile,epl_crowd,epl_noveh,epl_groupq,spl_theme4,rpl_theme4,spl_themes,rpl_themes,f_pov150,f_unemp,f_hburd,f_nohsdp,f_uninsur,f_theme1,f_age65,f_age17,f_disabl,f_sngpnt,f_limeng,f_theme2,f_minrty,f_theme3,f_munit,f_mobile,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_daypop,e_noint,m_noint,e_afam,m_afam,e_hisp,m_hisp,e_asian,m_asian,e_aian,m_aian,e_nhpi,m_nhpi,e_twomore,m_twomore,e_otherrace,m_otherrace,ep_noint,mp_noint,ep_afam,mp_afam,ep_hisp,mp_hisp,ep_asian,mp_asian,ep_aian,mp_aian,ep_nhpi,mp_nhpi,ep_twomore,mp_twomore,ep_otherrace,mp_otherrace
0,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000
1,10002,ZCTA5 10002,0.8223,76518,2894,39094,1241,36028,1326,27908,2853,2833,574,14688,1367,18301,1376,4074,766,17681,1287,10028,1549,9896,1062,2211,499,18393,1640,56964,3226,35725,1677,16,28,2461,449,29828,1403,2090,39,36.8000,3.5000,7.6000,1.4000,40.8000,3.5000,30.0000,2.0000,5.4000,1.0000,23.1000,1.7000,13.1000,1.8000,13.0000,1.4000,6.1000,1.4000,24.7000,2.0000,74.4000,3.1000,91.4000,3.2000,0.0000,0.1000,6.8000,1.2000,82.8000,1.8000,2.7000,0.1000,0.9148,0.7946,0.9219,0.9741,0.7207,4.3261,0.9639,0.7296,0.1831,0.5186,0.7390,0.9944,3.1647,0.8781,0.9369,0.9369,0.9369,0.9790,0.0000,0.9105,0.9915,0.7730,3.6540,0.9254,12.0817,0.9656,1,0,1,1,0,3,0,0,0,0,1,1,1,1,1,0,1,1,0,3,8,64307,8590,1110,6141,1194,19864,2190,28477,1989,74,83,24,45,1810,486,574,394,23.8000,2.9000,8.0000,1.5000,26.0000,2.5000,37.2000,2.2000,0.1000,0.1000,0.0000,0.1000,2.4000,0.6000,0.8000,0.5000


In [14]:
svi_df.shape

(204, 153)

In [15]:
# list(svi_df.columns)

In [16]:
svi_df.ep_nhpi.unique()

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

# **Step 2: SVI items**

q quick double check

In [17]:
link = "/content/drive/My Drive/X999/NewYork_ZCTA.csv"

In [18]:
svi_raw = pd.read_csv(link)
svi_raw.head(2)

,ST,STATE,ST_ABBR,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,36,New York,NY,6390,ZCTA5 06390,4.0467,53,39,253,49,19,19,17,16,0,13,9,26,0,13,27,34,0,13,6,11,31,33,0,18,9,53,20,51,0,18,4,5,0,18,0,13,17,16,32.1000,18.8000,0.0000,52.7000,47.4000,100.0000,0.0000,51.4000,50.9000,45.7000,0.0000,45.2000,11.3000,19.0000,58.5000,41.4000,0.0000,94.7000,17.0000,99.2000,37.7000,92.1000,0.0000,7.1000,1.6000,2.0000,0.0000,94.7000,0.0000,75.5000,32.1000,18.8000,0.8790,0.0000,0.9635,0.0000,0.9960,2.8385,0.6342,0.0000,0.1408,0.9944,0.0000,0.9775,2.1127,0.3009,0.8062,0.8062,0.8062,0.0000,0.4654,0.0000,0.0000,0.9735,1.4389,0.2205,7.1963,0.4192,0,0,1,0,1,2,0,0,1,0,1,2,0,0,0,0,0,0,1,1,5,601,9,14,0,13,9,19,0,13,0,13,8,16,3,7,0,13,47.4000,51.8000,0.0000,45.2000,17.0000,35.0000,0.0000,45.2000,0.0000,45.2000,15.1000,32.1000,5.7000,12.1000,0.0000,45.2000
1,36,New York,NY,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000


In [19]:
def is_nyc_zipcode(zipcode):
    zip_int = int(zipcode) if isinstance(zipcode, str) else zipcode

    # Manhattan: 10001-10282
    if 10001 <= zip_int <= 10282:
        return True
    # addition Manhattan: 10300-10499
    if 10300 <= zip_int <= 10499:
        return True
    # Bronx: 10451-10475
    if 10451 <= zip_int <= 10475:
        return True
    # Brooklyn: 11201-11256
    if 11201 <= zip_int <= 11256:
        return True
    # Queens: 11351-11436, 11101-11109
    if (11351 <= zip_int <= 11436) or (11101 <= zip_int <= 11109):
        return True
    # Staten Island: 10301-10314
    if 10301 <= zip_int <= 10314:
        return True
    # additional Queens ZIPs
    if zip_int in [11004, 11005, 11411, 11412, 11413, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429]:
        return True
    return False

In [20]:
nyc_df = svi_raw[svi_raw['FIPS'].apply(is_nyc_zipcode)]

In [21]:
nyc_df.shape

(204, 156)

In [22]:
nyc_df.EP_NHPI.unique()
# interesting, so there was an error in the source data

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

In [23]:
svi_raw.EP_NHPI.unique()

array([ 1.51e+01,  0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,
        8.00e-01,  1.20e+00,  5.00e-01,  4.00e-01,  1.10e+00,  7.00e-01,
        1.50e+00,  1.80e+00,  9.00e-01,  6.00e-01,  1.40e+00,  2.10e+00,
        2.30e+00,  1.00e+00])

In [24]:
-9.99e+02, 0.00e+00, 9.00e-01, 2.00e-01

(-999.0, 0.0, 0.9, 0.2)

# **Step 3: All boroughs and their eviction rates**

In [25]:
evictions_pre_post_mean = evictions_pre_post[['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']].mean()
evictions_pre_post_mean
# may need to merge ep_twomore and ep_otherrance toegther

,0
ep_afam,29.2346
ep_asian,8.9040
ep_hisp,38.7307
ep_nhpi,0.0115
ep_white,19.0236
ep_twomore,2.8025
ep_otherrace,1.0239


In [26]:
evictions_pre_post_mean = evictions_pre_post_mean.reset_index()

In [27]:
evictions_pre_post_mean.rename(columns = {'index':'race_svi', 0: "racial percentage"}, inplace=True)

In [28]:
evictions_pre_post_mean

,race_svi,racial percentage
0,ep_afam,29.2346
1,ep_asian,8.9040
2,ep_hisp,38.7307
3,ep_nhpi,0.0115
4,ep_white,19.0236
5,ep_twomore,2.8025
6,ep_otherrace,1.0239


In [29]:
# type(evictions_pre_post_mean)
# so this is correct

In [30]:
# evictions_pre_post.columns

In [31]:
neighbor_evictions = evictions_pre_post.groupby('nta').agg({'average_year_eviction_nta_count': 'mean', 'borough': 'first'}).reset_index()
neighbor_evictions.sort_values('average_year_eviction_nta_count', ascending=False, inplace=True)
neighbor_evictions

,nta,average_year_eviction_nta_count,borough
27,Central Harlem North-Polo Grounds,0.0109,MANHATTAN
182,Woodlawn-Wakefield,0.0074,BRONX
59,Flatbush,0.0039,BROOKLYN
38,Crown Heights North,0.0038,BROOKLYN
11,Bedford Park-Fordham North,0.0038,BRONX
178,Williamsbridge-Olinville,0.0037,BRONX
52,East Tremont,0.0037,BRONX
107,Mott Haven-Port Morris,0.0036,BRONX
169,Washington Heights South,0.0035,MANHATTAN
98,Marble Hill-Inwood,0.0034,MANHATTAN


In [32]:
man_nta = {
    'nta': neighbor_evictions['nta'].unique(),
    'eviction_rates': neighbor_evictions['average_year_eviction_nta_count']
}

man_nta_df = pd.DataFrame(man_nta)
man_nta_df

,nta,eviction_rates
27,Central Harlem North-Polo Grounds,0.0109
182,Woodlawn-Wakefield,0.0074
59,Flatbush,0.0039
38,Crown Heights North,0.0038
11,Bedford Park-Fordham North,0.0038
178,Williamsbridge-Olinville,0.0037
52,East Tremont,0.0037
107,Mott Haven-Port Morris,0.0036
169,Washington Heights South,0.0035
98,Marble Hill-Inwood,0.0034


In [33]:
avg_per_nta = neighbor_evictions.average_year_eviction_nta_count.mean()
avg_per_nta
# per building, per year, that's why it is similar to the borough one too

np.float64(0.0012530904589788123)

### **Step 3.2. All neighborhoods racial composite**

In [34]:
race_columns = ['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']
racial_avg_all = evictions_pre_post.groupby('nta')[['average_year_eviction_nta_count', 'ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace', 'rpl_themes']].mean()
racial_avg_all.sort_values('average_year_eviction_nta_count', ascending=False, inplace=True)
racial_avg_all.reset_index(inplace=True)
racial_avg_all

,nta,average_year_eviction_nta_count,ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes
0,Central Harlem North-Polo Grounds,0.0109,53.1706,3.8060,28.1355,0.0001,10.7591,3.0765,0.9779,0.9694
1,Woodlawn-Wakefield,0.0074,46.3838,2.3225,26.4032,0.0000,19.5334,2.7199,2.3837,0.9640
2,Flatbush,0.0039,50.3284,6.2411,12.8934,0.0136,25.4287,4.1145,0.9041,0.9270
3,Crown Heights North,0.0038,54.5405,2.4402,14.5104,0.0004,23.1759,4.2121,0.9645,0.9298
4,Bedford Park-Fordham North,0.0038,16.0227,3.1098,72.5584,0.0000,6.2559,1.4718,0.4192,0.9888
5,Williamsbridge-Olinville,0.0037,46.3329,4.3523,38.9846,0.0000,5.7140,2.2981,1.8550,0.9845
6,East Tremont,0.0037,28.0218,1.3305,64.7416,0.0000,3.1983,1.6115,0.7708,0.9962
7,Mott Haven-Port Morris,0.0036,25.1110,0.5096,69.6073,0.5501,2.6292,0.8451,0.5355,0.9984
8,Washington Heights South,0.0035,9.4778,4.0466,66.1568,0.0000,17.8351,1.9685,0.4406,0.9725
9,Marble Hill-Inwood,0.0034,6.8807,3.3681,65.9000,0.0000,20.7640,2.3974,0.4966,0.9602


In [35]:
racial_avg_all.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186
nta,Central Harlem North-Polo Grounds,Woodlawn-Wakefield,Flatbush,Crown Heights North,Bedford Park-Fordham North,Williamsbridge-Olinville,East Tremont,Mott Haven-Port Morris,Washington Heights South,Marble Hill-Inwood,University Heights-Morris Heights,Kew Gardens Hills,Mount Hope,East Concourse-Concourse Village,Hunters Point-Sunnyside-West Maspeth,Battery Park City-Lower Manhattan,East New York,Central Harlem South,Hudson Yards-Chelsea-Flatiron-Union Square,Belmont,Washington Heights North,Queens Village,Starrett City,East Flatbush-Farragut,Baisley Park,Lenox Hill-Roosevelt Island,Brownsville,West Concourse,Morrisania-Melrose,West Farms-Bronx River,West New Brighton-New Brighton-St. George,Hamilton Heights,Clinton,South Jamaica,Van Cortlandt Village,East Harlem North,Hunts Point,Rugby-Remsen Village,Kew Gardens,Prospect Lefferts Gardens-Wingate,Fordham South,Highbridge,Longwood,Crotona Park East,Bedford,Stuyvesant Heights,Mariner's Harbor-Arlington-Port Ivory-Granitev...,Norwood,Soundview-Bruckner,Seagate-Coney Island,Astoria,Ocean Hill,Bronxdale,Co-op City,Claremont-Bathgate,Jackson Heights,Richmond Hill,Kingsbridge Heights,Bushwick South,Soundview-Castle Hill-Clason Point-Harding Park,Melrose South-Mott Haven North,Hollis,Jamaica,Crown Heights South,Grymes Hill-Clifton-Fox Hills,Yorkville,East Harlem South,North Riverdale-Fieldston-Riverdale,Woodhaven,Port Richmond,SoHo-TriBeCa-Civic Center-Little Italy,Westchester-Unionport,Turtle Bay-East Midtown,South Ozone Park,Springfield Gardens North,Corona,East New York (Pennsylvania Ave),Eastchester-Edenwald-Baychester,West Village,Erasmus,Pelham Parkway,Old Astoria,Stapleton-Rosebank,Canarsie,Flushing,Lower East Side,Upper East Side-Carnegie Hill,Midtown-Midtown South,Pelham Bay-Country Club-City Island,Bay Ridge,St. Albans,Steinway,Spuyten Duyvil-Kingsbridge,Forest Hills,Upper West Side,Van Nest-Morris Park-Westchester Square,Flatlands,Schuylerville-Throgs Neck-Edgewater Park,Briarwood-Jamaica Hills,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Cypress Hills-City Line,Queensbridge-Ravenswood-Long Island City,Midwood,Rosedale,Murray Hill,Fort Greene,Clinton Hill,Bushwick North,North Side-South Side,College Point,Douglas Manor-Douglaston-Little Neck,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,Bensonhurst West,Manhattanville,Murray Hill-Kips Bay,East Village,Elmhurst,Ft. Totten-Bay Terrace-Clearview,Pomonok-Flushing Heights-Hillcrest,Parkchester,Rego Park,Bensonhurst East,Ridgewood,Brighton Beach,Springfield Gardens South-Brookville,Fresh Meadows-Utopia,East Elmhurst,Greenpoint,Cambria Heights,Ozone Park,Lincoln Square,New Brighton-Silver Lake,Homecrest,Gramercy,Chinatown,Bellerose,Jamaica Estates-Holliswood,Sunset Park West,East Williamsburg,Charleston-Richmond Valley-Tottenville,Morningside Heights,Kensington-Ocean Parkway,Sunset Park East,Bayside-Bayside Hills,Laurelton,Lindenwood-Howard Beach,Middle Village,Madison,Bath Beach,Dyker Heights,Borough Park,Woodside,Carroll Gardens-Columbia Street-Red Hook,Allerton-Pelham Gardens,Oakwood-Oakwood Beach,West Brighton,Gravesend,Maspeth,Old Town-Dongan Hills-South Beach,Park Slope-Gowanus,Great Kills,North Corona,Georgetown-Marine Park-Bergen Beach-Mill Basin,Grasmere-Arrochar-Ft. Wadsworth,Williamsburg,Prospect Heights,Stuyvesant Town-Cooper Village,Glendale,Oakland Gardens,Glen Oaks-Floral Park-New Hyde

## **This is for the race composites and neighborhoods bar char use**

# **Step 4 We want a dataframe that has neighborhoods as columns and average_year_eviction_nta_count as rows and contents**

In [36]:
evi_svi_df = evictions_pre_post.groupby('nta')[['average_year_eviction_nta_count','rpl_themes']].mean()
evi_svi_df.sort_values('average_year_eviction_nta_count', ascending=False, inplace=True)
evi_svi_df

,average_year_eviction_nta_count,rpl_themes
nta,,
Central Harlem North-Polo Grounds,0.0109,0.9694
Woodlawn-Wakefield,0.0074,0.9640
Flatbush,0.0039,0.9270
Crown Heights North,0.0038,0.9298
Bedford Park-Fordham North,0.0038,0.9888
Williamsbridge-Olinville,0.0037,0.9845
East Tremont,0.0037,0.9962
Mott Haven-Port Morris,0.0036,0.9984
Washington Heights South,0.0035,0.9725


In [37]:
evi_svi_df.reset_index(inplace=True)
evi_svi_df

,nta,average_year_eviction_nta_count,rpl_themes
0,Central Harlem North-Polo Grounds,0.0109,0.9694
1,Woodlawn-Wakefield,0.0074,0.9640
2,Flatbush,0.0039,0.9270
3,Crown Heights North,0.0038,0.9298
4,Bedford Park-Fordham North,0.0038,0.9888
5,Williamsbridge-Olinville,0.0037,0.9845
6,East Tremont,0.0037,0.9962
7,Mott Haven-Port Morris,0.0036,0.9984
8,Washington Heights South,0.0035,0.9725
9,Marble Hill-Inwood,0.0034,0.9602


In [38]:
avg_per_nta

np.float64(0.0012530904589788123)

In [39]:
average_svi = 0.80198
# https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

In [40]:
evi_svi_df['above_eviction_average'] = evi_svi_df['average_year_eviction_nta_count'] > avg_per_nta
evi_svi_df

,nta,average_year_eviction_nta_count,rpl_themes,above_eviction_average
0,Central Harlem North-Polo Grounds,0.0109,0.9694,True
1,Woodlawn-Wakefield,0.0074,0.9640,True
2,Flatbush,0.0039,0.9270,True
3,Crown Heights North,0.0038,0.9298,True
4,Bedford Park-Fordham North,0.0038,0.9888,True
5,Williamsbridge-Olinville,0.0037,0.9845,True
6,East Tremont,0.0037,0.9962,True
7,Mott Haven-Port Morris,0.0036,0.9984,True
8,Washington Heights South,0.0035,0.9725,True
9,Marble Hill-Inwood,0.0034,0.9602,True


In [41]:
evi_svi_df['above_svi_average'] = evi_svi_df['rpl_themes'] > average_svi
evi_svi_df

,nta,average_year_eviction_nta_count,rpl_themes,above_eviction_average,above_svi_average
0,Central Harlem North-Polo Grounds,0.0109,0.9694,True,True
1,Woodlawn-Wakefield,0.0074,0.9640,True,True
2,Flatbush,0.0039,0.9270,True,True
3,Crown Heights North,0.0038,0.9298,True,True
4,Bedford Park-Fordham North,0.0038,0.9888,True,True
5,Williamsbridge-Olinville,0.0037,0.9845,True,True
6,East Tremont,0.0037,0.9962,True,True
7,Mott Haven-Port Morris,0.0036,0.9984,True,True
8,Washington Heights South,0.0035,0.9725,True,True
9,Marble Hill-Inwood,0.0034,0.9602,True,True


# **Step 4.3 Run Chi-Square test**
  **Null Hypothesis: There is no association between neighborhoods that have above the average of black and hispanic percentage and have above than average svi scores**

In [42]:
contingency_table = pd.crosstab(evi_svi_df.above_svi_average, evi_svi_df.above_eviction_average)
contingency_table

above_eviction_average,False,True
above_svi_average,,
False,31,7
True,85,64


In [43]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(6.730235146153845),
 np.float64(0.009479210274470042),
 1,
 array([[23.57219251, 14.42780749],
        [92.42780749, 56.57219251]]))

### **P-value is 0.0095 and chi-square is 6.73. We can reject the null hypothesis. The Chi-square suggests a strong-to-moderate deviation from the distribution if the two groups/categories are independent.**

# **conclusion: there is statistically significant association between the two "above average eviction" and "above average svi scores" neighborhoods.**

In [44]:
a = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
b = evi_svi_df[(evi_svi_df['above_svi_average'] == True) & (evi_svi_df['above_eviction_average'] == False)].shape[0]
c = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == True)].shape[0]
d = evi_svi_df[(evi_svi_df['above_svi_average'] == False) & (evi_svi_df['above_eviction_average'] == False)].shape[0]

In [45]:
observed = np.array([[a, b], [c, d]])
observed

array([[64, 85],
       [ 7, 31]])

In [46]:
# a += 0.5
# b += 0.5
# c += 0.5
# d += 0.5
# # avoid 0 divisions

In [47]:
odds_ratio = (a * d) / (b * c)
odds_ratio
# extremely strong association between high svi and high evitction rates.

3.334453781512605

**This means, technically, neighborhoods with above average svi had 3.3345 times higher odds of having above average eviction rates compared to low-SVI neighborhoods.** This high odds may be due to all high svi neighborhoods had high evictions, and all high eviction neighborhoods had high svi from the sample.

In [48]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value

(np.float64(6.730235146153846), np.float64(0.009479210274470046))

# **Step 5: We also need a dataframe that has neighborhoods as columns and black and hispanics percentages as rows and contents**

In [49]:
racial_avg_all['black_hispanic_pct'] = racial_avg_all['ep_afam'] + racial_avg_all['ep_hisp']
racial_avg_all

,nta,average_year_eviction_nta_count,ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,black_hispanic_pct
0,Central Harlem North-Polo Grounds,0.0109,53.1706,3.8060,28.1355,0.0001,10.7591,3.0765,0.9779,0.9694,81.3062
1,Woodlawn-Wakefield,0.0074,46.3838,2.3225,26.4032,0.0000,19.5334,2.7199,2.3837,0.9640,72.7870
2,Flatbush,0.0039,50.3284,6.2411,12.8934,0.0136,25.4287,4.1145,0.9041,0.9270,63.2219
3,Crown Heights North,0.0038,54.5405,2.4402,14.5104,0.0004,23.1759,4.2121,0.9645,0.9298,69.0509
4,Bedford Park-Fordham North,0.0038,16.0227,3.1098,72.5584,0.0000,6.2559,1.4718,0.4192,0.9888,88.5811
5,Williamsbridge-Olinville,0.0037,46.3329,4.3523,38.9846,0.0000,5.7140,2.2981,1.8550,0.9845,85.3175
6,East Tremont,0.0037,28.0218,1.3305,64.7416,0.0000,3.1983,1.6115,0.7708,0.9962,92.7634
7,Mott Haven-Port Morris,0.0036,25.1110,0.5096,69.6073,0.5501,2.6292,0.8451,0.5355,0.9984,94.7183
8,Washington Heights South,0.0035,9.4778,4.0466,66.1568,0.0000,17.8351,1.9685,0.4406,0.9725,75.6346
9,Marble Hill-Inwood,0.0034,6.8807,3.3681,65.9000,0.0000,20.7640,2.3974,0.4966,0.9602,72.7807


In [50]:
evi_bh_df = racial_avg_all[['nta', 'average_year_eviction_nta_count', 'black_hispanic_pct']]
evi_bh_df

,nta,average_year_eviction_nta_count,black_hispanic_pct
0,Central Harlem North-Polo Grounds,0.0109,81.3062
1,Woodlawn-Wakefield,0.0074,72.7870
2,Flatbush,0.0039,63.2219
3,Crown Heights North,0.0038,69.0509
4,Bedford Park-Fordham North,0.0038,88.5811
5,Williamsbridge-Olinville,0.0037,85.3175
6,East Tremont,0.0037,92.7634
7,Mott Haven-Port Morris,0.0036,94.7183
8,Washington Heights South,0.0035,75.6346
9,Marble Hill-Inwood,0.0034,72.7807


In [51]:
average_bh_pct = evi_bh_df['black_hispanic_pct'].mean()
average_bh_pct

np.float64(49.541793265928135)

In [52]:
avg_per_nta, average_bh_pct
# evictions and black + hispanic pct

(np.float64(0.0012530904589788123), np.float64(49.541793265928135))

In [53]:
evi_bh_df['above_evi_average'] = evi_bh_df['average_year_eviction_nta_count'] > avg_per_nta
evi_bh_df

,nta,average_year_eviction_nta_count,black_hispanic_pct,above_evi_average
0,Central Harlem North-Polo Grounds,0.0109,81.3062,True
1,Woodlawn-Wakefield,0.0074,72.7870,True
2,Flatbush,0.0039,63.2219,True
3,Crown Heights North,0.0038,69.0509,True
4,Bedford Park-Fordham North,0.0038,88.5811,True
5,Williamsbridge-Olinville,0.0037,85.3175,True
6,East Tremont,0.0037,92.7634,True
7,Mott Haven-Port Morris,0.0036,94.7183,True
8,Washington Heights South,0.0035,75.6346,True
9,Marble Hill-Inwood,0.0034,72.7807,True


In [54]:
evi_bh_df['above_bh_average'] = evi_bh_df['black_hispanic_pct'] > average_bh_pct
evi_bh_df

,nta,average_year_eviction_nta_count,black_hispanic_pct,above_evi_average,above_bh_average
0,Central Harlem North-Polo Grounds,0.0109,81.3062,True,True
1,Woodlawn-Wakefield,0.0074,72.7870,True,True
2,Flatbush,0.0039,63.2219,True,True
3,Crown Heights North,0.0038,69.0509,True,True
4,Bedford Park-Fordham North,0.0038,88.5811,True,True
5,Williamsbridge-Olinville,0.0037,85.3175,True,True
6,East Tremont,0.0037,92.7634,True,True
7,Mott Haven-Port Morris,0.0036,94.7183,True,True
8,Washington Heights South,0.0035,75.6346,True,True
9,Marble Hill-Inwood,0.0034,72.7807,True,True


### **This is the neighborhood evictions + black/hispanic chi-test df:**

# **Run Chi-Square test**
  **Null Hypothesis: There is no association between neighborhoods that have above the average of black and hispanic percentage and have above than average svi scores**

In [55]:
contingency_table = pd.crosstab(evi_bh_df.above_bh_average, evi_bh_df.above_evi_average)
contingency_table

above_evi_average,False,True
above_bh_average,,
False,91,16
True,25,55


In [56]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(53.98752613823855),
 np.float64(2.0176646792192656e-13),
 1,
 array([[66.37433155, 40.62566845],
        [49.62566845, 30.37433155]]))

In [57]:
p_value

np.float64(2.0176646792192656e-13)

### **P-value is 0.00032 (extremely small) and chi-squiare is 53.9875. We can refute the null hypothesis. We can say we are confident that high "black+hispanic percetage" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high BH pct and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [58]:
a = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == True)].shape[0]
b = evi_bh_df[(evi_bh_df['above_bh_average'] == True) & (evi_bh_df['above_evi_average'] == False)].shape[0]
c = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == True)].shape[0]
d = evi_bh_df[(evi_bh_df['above_bh_average'] == False) & (evi_bh_df['above_evi_average'] == False)].shape[0]

In [59]:
observed = np.array([[a, b], [c, d]])
observed

array([[55, 25],
       [16, 91]])

In [60]:
odds_ratio = (a * d) / (b * c)
odds_ratio

12.5125

**This means, technically, neighborhoods with above average b+h pct had 12.5125 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.

In [61]:
chi2, p_value, dof, expected = chi2_contingency(observed)
chi2, p_value
# 11.32 is a strong deviation from expected undert the null hypothesis

(np.float64(53.98752613823855), np.float64(2.0176646792192656e-13))

# **Quick extreme cases**

In [86]:
evi_svi_df_top_15 = evi_svi_df.sort_values('average_year_eviction_nta_count', ascending=False).head(15)
# 2 of the bottom 10 were in cemetery areas
evi_svi_df_bottom_15 = evi_svi_df.sort_values('average_year_eviction_nta_count', ascending=False).tail(15)
evi_svi_df_extremes = pd.concat([evi_svi_df_top_15, evi_svi_df_bottom_15])
evi_svi_df_extremes

,nta,average_year_eviction_nta_count,rpl_themes,above_eviction_average,above_svi_average
0,Central Harlem North-Polo Grounds,0.0109,0.9694,True,True
1,Woodlawn-Wakefield,0.0074,0.9640,True,True
2,Flatbush,0.0039,0.9270,True,True
3,Crown Heights North,0.0038,0.9298,True,True
4,Bedford Park-Fordham North,0.0038,0.9888,True,True
5,Williamsbridge-Olinville,0.0037,0.9845,True,True
6,East Tremont,0.0037,0.9962,True,True
7,Mott Haven-Port Morris,0.0036,0.9984,True,True
8,Washington Heights South,0.0035,0.9725,True,True
9,Marble Hill-Inwood,0.0034,0.9602,True,True


In [87]:
contingency_table = pd.crosstab(evi_svi_df_extremes.above_svi_average, evi_svi_df_extremes.above_eviction_average)
contingency_table

above_eviction_average,False,True
above_svi_average,,
False,5,1
True,10,14


In [88]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(1.875),
 np.float64(0.17090352023079358),
 1,
 array([[ 3.,  3.],
        [12., 12.]]))

### **P-value is 0.0935 (small) and chi-squiare is 2.8125. We can refute the null hypothesis. We can say we are confident that high "svi" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high SVI and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [94]:
a = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == True) & (evi_svi_df_extremes['above_eviction_average'] == True)].shape[0]
b = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == True) & (evi_svi_df_extremes['above_eviction_average'] == False)].shape[0]
c = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == False) & (evi_svi_df_extremes['above_eviction_average'] == True)].shape[0]
d = evi_svi_df_extremes[(evi_svi_df_extremes['above_svi_average'] == False) & (evi_svi_df_extremes['above_eviction_average'] == False)].shape[0]

In [96]:
observed = np.array([[a, b], [c, d]])
observed

array([[14, 10],
       [ 1,  5]])

In [95]:
# a += 0.5
# b += 0.5
# c += 0.5
# d += 0.5

In [97]:
odds_ratio = (a * d) / (b * c)
odds_ratio

7.0

**This means, technically, neighborhoods with above average b+h pct had 21 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.

In [98]:
evi_bh_df_top_15 = evi_bh_df.sort_values('average_year_eviction_nta_count', ascending=False).head(15)
evi_bh_df_bottom_15 = evi_bh_df.sort_values('average_year_eviction_nta_count', ascending=False).tail(15)
evi_bh_df_extremes = pd.concat([evi_bh_df_top_15, evi_bh_df_bottom_15])
evi_bh_df_extremes

,nta,average_year_eviction_nta_count,black_hispanic_pct,above_evi_average,above_bh_average
0,Central Harlem North-Polo Grounds,0.0109,81.3062,True,True
1,Woodlawn-Wakefield,0.0074,72.7870,True,True
2,Flatbush,0.0039,63.2219,True,True
3,Crown Heights North,0.0038,69.0509,True,True
4,Bedford Park-Fordham North,0.0038,88.5811,True,True
5,Williamsbridge-Olinville,0.0037,85.3175,True,True
6,East Tremont,0.0037,92.7634,True,True
7,Mott Haven-Port Morris,0.0036,94.7183,True,True
8,Washington Heights South,0.0035,75.6346,True,True
9,Marble Hill-Inwood,0.0034,72.7807,True,True


In [99]:
contingency_table = pd.crosstab(evi_bh_df_extremes.above_bh_average, evi_bh_df_extremes.above_evi_average)
contingency_table

above_evi_average,False,True
above_bh_average,,
False,14,2
True,1,13


In [100]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
chi2, p_value, dof, expected

(np.float64(16.205357142857142),
 np.float64(5.683316915964556e-05),
 1,
 array([[8., 8.],
        [7., 7.]]))

### **P-value is 0.00032 (extremely small) and chi-squiare is 12.929. We can refute the null hypothesis. We can say we are confident that high "black+hispanic" boroughs are more likely to have high evictions (or vice versa) or there is a strong association between high black+hispanic and high eviction rates in the full dataset**

# **conclusion: there is a statistically significant association between the two "above average eviction" and "above average black+hispanic pct" neighborhoods.**

In [104]:
a = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == True) & (evi_bh_df_extremes['above_evi_average'] == True)].shape[0]
b = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == True) & (evi_bh_df_extremes['above_evi_average'] == False)].shape[0]
c = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == False) & (evi_bh_df_extremes['above_evi_average'] == True)].shape[0]
d = evi_bh_df_extremes[(evi_bh_df_extremes['above_bh_average'] == False) & (evi_bh_df_extremes['above_evi_average'] == False)].shape[0]

In [105]:
observed = np.array([[a, b], [c, d]])
observed

array([[13,  1],
       [ 2, 14]])

In [103]:
# a += 0.5
# b += 0.5
# c += 0.5
# d += 0.5

In [106]:
odds_ratio = (a * d) / (b * c)
odds_ratio

91.0

**This means, technically, neighborhoods with above average b+h pct had 45 times higher odds of having above average eviction rates compared to low-bh-pct neighborhoods in nyc.** This high odds may be due to all high bh pct neighborhoods had high evictions, and all high eviction neighborhoods had high bh pct from the sample.